# Project Description : 
The ammount of aviation transportation is increasing day by day.
Flights number increases, Airplanes model and categories . Destinations all over the world is reachable !
And this is a common topic , and huge field , which all of us could work with !
In our neo4j project , we choosed to use a database of an api called "aviation-edge"  , available on " https://aviation-edge.com/ "
The objective of this project is to manipulate datasets , and explore them to get insights from them using what we learned in neo4j class!
It is a good experience to practice this tool by trying to model , mix and query a different data , and struggle with importing and setting.



### I -DATA SETS USED :

1) Airport data
https://aviation-edge.com/v2/public/airportDatabase?key=3abaa5-51c412
2) Flights data
https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412
3) Airlines data
https://aviation-edge.com/v2/public/airlineDatabase?key=3abaa5-51c412
4) Country data
https://aviation-edge.com/v2/public/countryDatabase?key=3abaa5-51c412
5) Airlplanes data
https://aviation-edge.com/v2/public/airplaneDatabase?key=3abaa5-51c412



#### II - IMPORT DATA AND CREATION (NODES & REALTIONS) :

###### Airport Data

In [ ]:

 WITH "https://aviation-edge.com/v2/public/airportDatabase?key=3abaa5-51c412" AS ap
CALL apoc.load.json(ap) YIELD value 
MERGE(a:AIRPORT {a_id:value.airportId}) ON CREATE
SET a.latitude= toFloat(value.latitudeAirport), a.longitude= toFloat(value.longitudeAirport), a.names= value.codeIataAirport, a.country= value.nameCountry, a.name= value.nameAirport, a.codeCountry = value.codeIso2Country
MERGE (co:Country_Name {Name: a.country})
MERGE (vi:Code_Airport {Name: a.names})
MERGE (la:Latitude_Airport {Number: a.latitude})
MERGE (lo:Longitude_Airport {Number: a.longitude})
CREATE (vi)-[:IN]->(co)
CREATE (la)-[:LAT]->(vi)
CREATE (lo)-[:LONG]->(vi)

###### Flights Data

In [ ]:
CREATE CONSTRAINT node_plane ON (f:FLIGHTS) ASSERT (f.Name) IS NODE KEY

WITH "https://aviation-edge.com/v2/public/flights?key=3abaa5-51c412" AS vol
CALL apoc.load.json(vol) YIELD value 
UNWIND value.airline AS cie 
UNWIND value.aircraft AS plane
UNWIND value.geography  AS geo
UNWIND value.speed AS speeds
UNWIND value.arrival AS arrival
UNWIND value.departure AS departure
MERGE(f:FLIGHTS {f_id:plane.regNumber}) ON CREATE
SET f.Name= plane.regNumber, f.Status= value.status, f.Company_Code= cie.icaoCode, f.Latitude= toFloat(geo.latitude), f.Longitude= toFLoat(geo.longitude), f.Speed = toFloat(speeds.horizontal), f.Arrival = arrival.iataCode, f.Departure = departure.iataCode
FOREACH(uselessVar IN CASE WHEN trim(f.Name) <> "" THEN [1] ELSE [] END | 
MERGE (pc:Plane {Code: f.Name})
MERGE (la:Latitude_Plane {Number: f.Latitude})
MERGE (lo:Longitude_Plane {Number: f.Longitude})
MERGE (sp:Plane_Speed {Code: f.Speed})
MERGE (st:Plane_Status {Status: f.Status})
MERGE (cy:Plane_Company {Name: f.Company_Code})
MERGE (ar:Arrivals {Code: f.Arrival})
MERGE (de:Departures {Code: f.Departure})
CREATE (la)-[:LAT]->(pc)
CREATE (lo)-[:LONG]->(pc)
CREATE (sp)-[:SPEED]->(pc)
CREATE (pc)-[:STATUS]->(st)
CREATE (pc)-[:BELONGS_TO]->(cy)
CREATE (de)-[:MOVE_TO]->(ar))

##### Airlines Data

In [ ]:
WITH 'https://aviation-edge.com/v2/public/airlineDatabase?key=3abaa5-51c412' AS airline
CALL apoc.load.json(airline) YIELD value
MERGE (a:AIRLINES {a_id: value.airlineId}) ON CREATE
SET a.Name= value.nameAirline, a.Code= value.codeIcaoAirline, a.Status= value.statusAirline, a.CodeCountry= value.codeIso2Country
MERGE (na: Airline_Name {Name: a.Name})
MERGE (co: Airline_Code {Code: a.Code})
CREATE (co)-[:IS]->(na)

###### Country data

In [ ]:
WITH 'https://aviation-edge.com/v2/public/countryDatabase?key=3abaa5-51c412' AS country
CALL apoc.load.json(country) YIELD value
MERGE (c:COUNTRIES {c_id: value.countryId}) ON CREATE
SET c.Currency_Code = value.codeCurrency, c.Currency= value.nameCurrency, c.Code= value.codeIso2Country, c.Name= value.nameCountry
MERGE (na: Country_Name {Name: c.Name})
MERGE (cu: Currency {Code: c.Currency_Code})
CREATE (na)-[:PAY_WITH]->(cu)


###### Airplanes data 


In [ ]:
WITH 'https://aviation-edge.com/v2/public/airplaneDatabase?key=3abaa5-51c412' AS airplane
CALL apoc.load.json(airplane) YIELD value
MERGE (c:AIRPLANE {a_id: value.airplaneId}) ON CREATE
SET c.Date = value.registrationDate, c.Model = value.planeModel, c.Code = value.numberRegistration


## QUERY DATA 

After visualizing datastes, many questions and ideas could be asked and explored.
So here , there are some usefull aqueries .


1°Q : what are the top 5 companies with most airplanes in the air at the moment?

In [ ]:

MATCH (q:FLIGHTS)
WHERE q.Status = 'en-route' AND NOT q.Company_Code = ""
RETURN count(q.Status) AS Total, q.Company_Code AS Company ORDER BY Total DESC LIMIT 5

2°Q : To which companies belongs the codes of most airplanes in the air?

In [ ]:
MATCH (a1:Plane_Company), (a2:Airline_Code)
WHERE a1.Name = a2.Code
CREATE (a1)-[:SAME]->(a2)

3°Q : How many flights are there with one stop from a given origin to a given destination?
With :
- most of them in arrival situation :

In [ ]:
#more arrival
MATCH (f:FLIGHTS), (a:AIRPORT)
WHERE f.Arrival = a.names
RETURN a.name,  count(f.Arrival)  AS Total ORDER BY Total DESC LIMIT 5


- most of them in departure situation:

In [ ]:
#more departure
MATCH (f:FLIGHTS), (a:AIRPORT)
WHERE f.Departure = a.names
RETURN a.name,  count(f.Departure)  AS Total ORDER BY Total DESC LIMIT 5

- with more trafic:

In [ ]:

#more trafic 
MATCH (f:FLIGHTS), (a:AIRPORT)
WHERE f.Departure = a.names OR  f.Arrival = a.names
RETURN a.name,  (count(f.Departure)+count(f.Arrival))/2  AS Total ORDER BY Total DESC LIMIT 5

- with more departure situation per country:

In [ ]:

#more departure per country
MATCH (f:FLIGHTS), (a:AIRPORT), (c:COUNTRIES)
WHERE f.Departure = a.names  AND a.codeCountry = c.Code
RETURN c.Name, count(f.Departure) AS Total ORDER BY Total DESC LIMIT 5

- with more arrival situation per country:

In [ ]:
#more arrival per country
MATCH (f:FLIGHTS), (a:AIRPORT), (c:COUNTRIES)
WHERE f.Arrival = a.names  AND a.codeCountry = c.Code
RETURN c.Name, count(f.Arrival) AS Total ORDER BY Total DESC LIMIT 5


4°Q : Which are the 3 Countries with Most Airlines from our given/available dataset?

In [ ]:
MATCH (country:Country )<-[:BELONGS_TO]-(airline:Airline) 
WITH country,count(DISTINCT airline) AS airline_count ORDER BY airline_count DESC 
RETURN country.countryName,airline_count LIMIT 3

5°Q : What are the top 10 country which have an active airline statut ?

In [ ]:
#Maybe do it with a relation
MATCH (c:COUNTRIES), (a:AIRLINES)
WHERE c.Code = a.CodeCountry AND a.Status = 'active'
RETURN c.Name, count(a.Code) AS NumberOfAirlines ORDER BY NumberOfAirlines DESC LIMIT 10

6°Q : WHat are Top 2 Airlines with Most Flights?

In [ ]:
MATCH (airline:Airline)<-[r]-(f:Flight) 
WITH airline, count(r) AS flight_count ORDER BY flight_count DESC 
RETURN airline.airlineName, flight_count LIMIT 2

Since we are in France , we are more interested in countries who have same currency !

7°Q : What is Number of airport which currency used is euro?
#and what are their country?

In [ ]:
#MATCH (airp: nameAirport) WHERE nameCurrency = Euro 
#RETURN count airp.nameAirport AS NbrAirportCurEuro
MATCH (a:AIRPORT), (c:COUNTRIES)
WHERE a.codeCountry = c.Code
RETURN c.Currency_Code AS Currency, count(a.name) AS NumberOfAirports ORDER BY NumberOfAirports DESC LIMIT 5



Now , to be more curious , someone asked :


8°Q : Which  Airplane owner has most of active airplanes?


In [ ]:
MATCH (owner : planeOwner) 
count (owner) WHERE statut = active 
RETURN owner.planeOwner ORDER BY DESC LIMIT 1


And it is interesting to know this question :

9°Q:  What are Planes on air flying closeby a chosen airport?


In [ ]:
MATCH (f:FLIGHTS), (a:AIRPORT)
WITH a.latitude AS lata, a.longitude AS longa, f.Longitude AS longf,f.Latitude AS latf, f.Status AS Status, f.Name AS numb
WHERE a.names = 'AAC' AND Status = 'en-route'
WITH point( {longitude:longf,latitude: latf} ) AS point1, point( {longitude:longa,latitude: lata} ) AS point2, numb AS numb
RETURN numb AS Plane_Reference, round(distance(point1,point2)/1000) AS Distance_kilometers ORDER BY Distance_kilometers  ASC LIMIT 5

And for who is fun of weather features :


10°Q : What are Airport with temperature, pressure and wind ?

In [ ]:
WITH "https://api.openweathermap.org/data/2.5/weather?lat=latitude&lon=longitude&appid=5842eebcfff240981095a321de43a2fa" AS meteo, '-50' AS lat_input
WITH apoc.text.replace(meteo,'latitude',lat_input) AS meteo2, '20' AS long_input
WITH apoc.text.replace(meteo2,'longitude',long_input) AS meteo3
CALL apoc.load.json(meteo3) YIELD value
UNWIND value.main AS attribute
UNWIND value.wind AS attribute2
RETURN attribute.temp AS Temperature, attribute2.speed AS Speed, attribute.pressure AS Pressure